In [ ]:
import gym
from gym import wrappers

In [ ]:
env = gym.make('SpaceInvaders-v0')
env.reset()
env = wrappers.Monitor(env, 'SpaceInvaders-v0', force=True)
for i_episode in range(105):
    observation = env.reset()
    while (True):
        env.render()
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        if done:
            print("Episode finished after {} episodes".format(i_episode+1))
            break

env.close()
#gym.upload('Pong-v0-1', api_key='sk_WRCITkqmTJKYB9hvBk5tPA')

In [ ]:
env.close()

In [2]:
import numpy as np
import random
from collections import deque
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten
from keras.optimizers import Adam


class Agent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01  # exploration will not decay futher
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()
        self.weight_backup = 'model_weights.h5'
    '''
    def _build_model(self):
        model = Sequential(),
        model.add(Conv2D(32, kernel_size=8, subsample=(4, 4), activation='relu', padding='same', input_shape=4))  #80*80*4
        model.add(Conv2D(64, kernel_size=4, subsample=(2, 2), activation='relu', padding='same'))
        model.add(Conv2D(64, kernel_size=3, subsample=(1, 1), activation='relu', padding='same'))
        model.add(Flatten())
        model.add(Dense(512, activation='relu'))
        model.add(Dense(2, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model
    '''

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def save_model(self):
            self.model.save(self.weight_backup)

    def act(self, state):
        if np.random.rand() <= self.exploration_rate:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def remember(self, state, action, reward, new_state, done):
        self.memory.append((state, action, reward, new_state, done))

    def memory_replay(self, batch_size):
        if len(self.memory) < batch_size:
            return
            sample = random.sample(self.memory, batch_size)
        for state, action, reward, new_state, done in sample:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(new_state)[0])
                target_f = self.model.predict(state)
                target_f[0][action] = target
                self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.exploration_rate > self.exploration_min:
            self.exploration_rate *= self.exploration_decay


Using TensorFlow backend.


In [9]:
env.observation_space.shape

(4,)

In [10]:
env.action_space.n

2

In [4]:
import sys
import gym
from gym import wrappers

batch_size = 32
#episodes = sys.argv[1] if len(sys.argv) > 1 else 5000
#env_name = sys.argv[2] if len(sys.argv) > 2 else "Pong-v0"

episodes = 5000
env_name = "CartPole-v1"

env = gym.make(env_name)
env.reset()
agent = Agent(env.observation_space.shape[0], env.action_space.n)

env = wrappers.Monitor(env, env_name, force=True)
for i_episodes in range(episodes):
    state = env.reset()
    state = np.reshape(state, [1, env.observation_space.shape[0]])
    while (True):
        env.render()
        action = agent.act(state)
        new_state, reward, done, info = env.step(action)
        agent.remember(state, action, reward, new_state, done)
        state = new_state
    agent.memory_replay(batch_size)
    if done:
        print("Game finished after {} episodes".format(i_episodes + 1))
        agent.save_model()
        break

env.close()

[2017-08-12 13:34:55,862] Making new env: CartPole-v1
/home/z0m6ie/Documents/GitHub/Machine_Learning_Projects/deep-Q-learning/agent.py:24: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, kernel_size=8, activation="relu", padding="same", input_shape=4, strides=(4, 4))`
  model.add(Conv2D(32, kernel_size=8, subsample=(4, 4), activation='relu', padding='same', input_shape=self.state_size))  #80*80*4


TypeError: 'int' object is not iterable